# **Introduction: `Context Engineering`**

**`“Context Engineering”`** is a newer, broader idea in the world of prompt-based / agent-based AI systems. It builds on prompt engineering, but adds system design around *what* the model sees, *how* it evolves, and *what* gets kept or removed. I’ll explain what it is, how it works, key strategies, and a concrete example.



## **What is Context Engineering:**

From the sources:

* According to DataCamp: <u>*“Context engineering is the practice of designing systems that decide what information an AI model sees before it generates a response.”* </u>
* According to LangChain (blog): it’s <u>*“the art and science of filling the context window with just the right information at each step of an agent’s trajectory.”*</u>

So it’s about:

1. **Selecting** which kinds of context are relevant (instructions, memory, tools, retrieved external knowledge, etc.).
2. **Managing** that context as things evolve (over multiple steps, tool calls, long-running conversation) so that the model doesn’t get swamped, confused, or distracted.
3. **Formatting** or organizing the context so it's helpful and usable to the model.
4. **Persisting / retrieving / compressing** past information when needed.



## **How It Works — Key Components & Strategies:**

From LangChain’s blog, there are 4 major strategy categories for agent / long-running systems: **Write, Select, Compress, Isolate**. 

Here’s a breakdown:

| Strategy     | Purpose                                                                                                                      | Techniques / Examples                                                                                                                                                                                       |
| ------------ | ---------------------------------------------------------------------------------------------------------------------------- | ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **Write**    | Save context externally (outside the model’s window) so it can be retrieved later — avoid overloading the immediate context. | Use *scratchpads*, *external memory stores*, logs of tool feedback; write summaries of past interactions so important info isn’t lost.                                                 |
| **Select**   | Among all the possible context, choose what’s most relevant to bring in.                                                     | Retrieval Augmented Generation (RAG) where you pick relevant docs; memory systems that pick recent & pertinent memory; filtering history; selecting most relevant system/tool instructions. |
| **Compress** | Shrink context (summarization / abstraction) so you can keep core ideas without exceeding context limits.                    | Summarizing long tool call feedback; compressing conversation history; generating abstractions or meta-memories. [2])                                                                      |
| **Isolate**  | Separating context types or keeping “clean” contexts so that irrelevant noise or conflicting info don’t degrade performance. | Running sub-agents with their own context, sandboxing parts of context, isolating tool outputs, having separate memory for different tasks or topics.                               |

Also, from DataCamp: context engineering involves multiple **types of context** that need to be managed: `system instructions`, `conversation history & preferences`, `retrieved documents`, `available tools` & `their definitions`, `real-time API data`, etc. 



## **When Context Engineering vs Prompt Engineering:**

* Prompt engineering is about *how you phrase* a particular input/task. One‐shot instructions etc.
* Context engineering is at a larger scale: how do you provide *all* necessary information over time, how do you manage memory & tools, how do you adapt context as task evolves. 



## **A Concrete Example:**

Let’s build an example to illustrate how context engineering helps:

### **Scenario**

Suppose you are building a customer support AI agent. The agent supports users over multiple sessions, can pull up product info, past tickets, user preferences, and perform tasks that require external APIs (e.g. check order status, initiate refund).

### **Without good context engineering**

* Every time user contacts, you send the system prompt + latest user query + tool descriptions.
* No memory of past tickets / preferences → user has to repeat things.
* If the conversation gets long, the agent’s context window may overflow; older context may push out important instructions or user preferences.
* Tool feedback (what the APIs returned before) may clutter the conversation history, confusing the agent.

### **With context engineering**

You would design the system architecture to include:

1. **Memory / Persistence (Write)**

   * Keep long-term memory of user profile: preferences, past orders, issues resolved.
   * Keep scratchpad of ongoing conversation (short term).

2. **Selection**

   * For each new user query, retrieve from memory only the parts relevant: maybe if they ask about a product, fetch past tickets about that product.
   * Only carry latest few turns of conversation in immediate context.

3. **Compression**

   * Summarize past resolved tickets or long conversations into compact summaries (e.g., “User prefers email over chat,” or “Issue with defective parts last time”)
   * Compress tool feedback; don’t send full JSON output if only a part is needed.

4. **Isolation**

   * Separate contexts: e.g. order help context, billing context, product info context.
   * Use sub-agents for specialized tasks (one that handles refunds, another that handles technical issues), each has its own limited relevant memory or context.

5. **Formatting / Tools & System Instructions**

   * Have system instructions that define policies (e.g. tone, privacy).
   * Include definitions of tools the agent can call (APIs) so model knows how to use them.
   * Format how retrieved documents are included (e.g. “Here is relevant product spec: …”) vs how privileges or preferences are expressed.

### **What happens in practice**

When user comes back with: “Hi, it’s me again. I had a problem with the XYZ model; it didn’t charge fully. Also I want a refund.”

* The system pulls up their profile (knows they prefer email).
* It retrieves past tickets related to model XYZ.
* It retrieves tool definitions for refund APIs.
* It compresses old conversations so only summaries are in context.
* Immediate past few user/agent turns are in context for flow.
* The system prompt includes instructions about tone, privacy, etc.

This keeps the prompt clear, relevant, not flooded with unnecessary info, and enables the agent to respond appropriately (e.g. apologize, fetch order status, initiate refund) without confusion or repeating questions.



## **Benefits & Risks**

**Benefits:**

* Better relevance and accuracy (agent has what it needs).
* Improved user experience (less repetition, more coherence).
* Efficiency: avoid sending huge contexts → cost & latency savings.
* Scaling: systems that work over many turns, with many tools, many users.

**Risks / Challenges:**

* Designing what is relevant vs what isn’t can be tricky.
* Irrelevant context or outdated memory can hurt more than help ("context poisoning").
* Deciding when to compress vs drop info.
* Privacy/security: memory & context may include sensitive info. Care needed.
* Complexity: requires infrastructure (memory stores, retrieval, summarization) rather than just writing prompts.


![context_eng](context_eng.png)

## **What happen if Context Engineering not handle properly:**

Excellent follow-up 🙌 — these four are the **failure modes** that can occur when we do Context Engineering poorly. Let’s go through them one by one with **clear examples** so you can see how they differ.


### **⚠️ 1. Context Poisoning**

* **Definition:** `Irrelevant`, `outdated`, or `malicious information` sneaks into the model’s context and corrupts its outputs.
  * Think of it as *“garbage in → garbage out.”*

* **Example:**
    You have an AI agent helping with medical advice. Its memory incorrectly stores:

    > “Aspirin cures all viral infections.”
    > Now every time the user asks about colds or flu, the agent might confidently recommend aspirin — dangerous because poisoned context outweighed the real retrieval sources.

    👉 Context poisoning is often a **security risk** (malicious injection) or a **data hygiene risk** (bad or outdated info persisting in memory).


### **⚠️ 2. Context Distraction**

* **Definition:** Too much `irrelevant information` floods the context window, distracting the model from what’s important.
  * Think of it as *“signal drowned by noise.”*

* **Example:**
    User: *“What is the warranty period for iPhone 15 Pro?”*
    Context given to model:

      * 5 pages of unrelated product manuals
      * Customer reviews of old iPhone models
      * Warranty details for iPads
      * **One line** about “iPhone 15 Pro warranty: 1 year”

    The model might focus on iPads or iPhone 14 details and miss the one correct line → produces wrong answer.



### **⚠️ 3. Context Confusion**

* **Definition:** Different parts of the context give **conflicting signals**, leaving the model uncertain which to trust.
  * Think of it as *“contradictions inside the context.”*

* **Example:**

  * Memory summary says: “User prefers refund via **PayPal**.”
  * Latest tool output says: “Refund method: **Credit Card**.”
  * System instruction says: “Always use **store credit** by default.”

    Now the agent is confused: should it issue PayPal refund, credit card refund, or store credit?
    → This can cause errors or unpredictable answers.



### **⚠️ 4. Context Clash**

* **Definition:** When **different context layers** (`system prompt`, `user prompt`, `retrieved documents`, `memory`) **conflict with each other** and the model doesn’t know which one takes precedence.
  * Think of it as *“authority conflict between layers.”*

* **Example:**
  * **System prompt:** “Always be formal and polite.”
  * **Memory context:** “User prefers casual tone.”
  * **User prompt:** “Please explain this in a super casual way.”

    The model is caught in a **clash**: should it obey the system rule, the user’s immediate instruction, or the saved preference?
    → If not carefully engineered (with rules about precedence), outputs can be inconsistent.



### **🔑 Quick Summary**

| Failure Mode            | Root Cause                                           | Example Symptom                        |
| ----------------------- | ---------------------------------------------------- | -------------------------------------- |
| **Context Poisoning**   | Bad/malicious info injected                          | Model gives harmful or outdated advice |
| **Context Distraction** | Too much irrelevant info                             | Model misses the right answer          |
| **Context Confusion**   | Contradictory details in same layer                  | Model gives inconsistent responses     |
| **Context Clash**       | Different layers (system vs user vs memory) conflict | Model flip-flops tone or behavior      |